## test code part 1

In [1]:
from langchain.schema.messages import HumanMessage, SystemMessage
from langchain_intro.chatbot import chat_model

In [2]:
messages = [
    SystemMessage(
        content="""You're an assistant knowledgeable about
        healthcare. Only answer healthcare-related questions."""
    ),
    HumanMessage(content="What is Medicaid managed care?"),
]

In [3]:
chat_model.invoke(messages)

AIMessage(content='Medicaid managed care is a system in which state Medicaid programs contract with private health insurance companies to provide healthcare services to Medicaid beneficiaries. In this model, the managed care organizations (MCOs) are responsible for coordinating and delivering a range of healthcare services, including preventive care, hospital services, and specialty care, to enrolled members.\n\nKey features of Medicaid managed care include:\n\n1. **Care Coordination**: MCOs help coordinate care among various providers to ensure that beneficiaries receive appropriate and timely services.\n\n2. **Cost Control**: By managing the care provided to beneficiaries, MCOs aim to control costs and improve efficiency in the delivery of healthcare services.\n\n3. **Access to Services**: Beneficiaries typically have a network of providers from which to choose, and MCOs may offer additional services beyond what traditional Medicaid covers.\n\n4. **Quality Improvement**: Many MCOs im

In [4]:
from langchain.prompts import ChatPromptTemplate

review_template_str = """Your job is to use patient
reviews to answer questions about their experience at a hospital.
Use the following context to answer questions. Be as detailed
as possible, but don't make up any information that's not
from the context. If you don't know an answer, say you don't know.

{context}

{question}
"""

review_template = ChatPromptTemplate.from_template(review_template_str)

context = "I had a great stay!"
question = "Did anyone have a positive experience?"

review_template.format(context=context, question=question)

"Human: Your job is to use patient\nreviews to answer questions about their experience at a hospital.\nUse the following context to answer questions. Be as detailed\nas possible, but don't make up any information that's not\nfrom the context. If you don't know an answer, say you don't know.\n\nI had a great stay!\n\nDid anyone have a positive experience?\n"

In [5]:
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)

review_system_template_str = """Your job is to use patient
reviews to answer questions about their experience at a
hospital. Use the following context to answer questions.
Be as detailed as possible, but don't make up any information
that's not from the context. If you don't know an answer, say
you don't know.

{context}
"""

review_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["context"], template=review_system_template_str
    )
)

review_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["question"], template="{question}"
    )
)

messages = [review_system_prompt, review_human_prompt]
review_prompt_template = ChatPromptTemplate(
    input_variables=["context", "question"],
    messages=messages,
)
context = "I had a great stay!"
question = "Did anyone have a positive experience?"

review_prompt_template.format_messages(context=context, question=question)

[SystemMessage(content="Your job is to use patient\nreviews to answer questions about their experience at a\nhospital. Use the following context to answer questions.\nBe as detailed as possible, but don't make up any information\nthat's not from the context. If you don't know an answer, say\nyou don't know.\n\nI had a great stay!\n"),
 HumanMessage(content='Did anyone have a positive experience?')]

## test code part 2

In [1]:
from langchain_intro.chatbot import review_chain

context = "I had a great stay!"
question = "Did anyone have a positive experience?"

review_chain.invoke({"context": context, "question": question})

AIMessage(content='Yes, one patient mentioned that they had a great stay at the hospital, indicating a positive experience.', response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 83, 'total_tokens': 103}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-9e440f5f-9648-454b-b1c7-1f4e99719b65-0')

## test code 3

In [7]:
import pandas as pd
from uuid import uuid4

from langchain_elasticsearch import ElasticsearchStore
from langchain_core.documents import Document
from elasticsearch import Elasticsearch

import os
import dotenv
import sentence_transformers

dotenv.load_dotenv(encoding='utf-8')

# File paths and other configurations
REVIEWS_CSV_PATH = "data/reviews.csv"
ELASTICSEARCH_URL = "http://localhost:9200"
INDEX_NAME = "reviews_index"

# Delete the index if exists
# Initialize Elasticsearch client
es_client = Elasticsearch(
    hosts=[ELASTICSEARCH_URL],  
    http_auth=("elastic", "changeme")  
)

# Check if the index exists, and delete it if it does
es_client.indices.delete(index=INDEX_NAME, ignore_unavailable=True)

# Load CSV data with pandas, ensuring UTF-8 encoding
df = pd.read_csv(REVIEWS_CSV_PATH, encoding='utf-8')

# Configure ElasticsearchStore (OpenAI embedding is too big )
elastic_vector_search = ElasticsearchStore(
    es_url=ELASTICSEARCH_URL,
    index_name=INDEX_NAME,
    es_user="elastic",
    es_password="changeme",
    strategy=ElasticsearchStore.ApproxRetrievalStrategy(
        query_model_id="sentence_transformers/all-MiniLM-L6-v2"
    ),
)

# Function to create documents with UUID, content, and metadata
def create_documents(df):
    # Create Document objects
    documents = []
    uuids = []

    for _, row in df.iterrows():
        doc_id = str(uuid4())
        documents.append(Document(
            page_content=row["review"],  # Use the "review" column for content
            metadata={
                "review_id": row["review_id"],
                "visit_id": row["visit_id"],
                "physician_name": row["physician_name"],
                "hospital_name": row["hospital_name"],
                "patient_name": row["patient_name"]
            }
        ))
        uuids.append(doc_id)

    return documents, uuids

/tmp/ipykernel_39216/763999832.py:21: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es_client = Elasticsearch(


In [8]:
# Create a list of documents from the DataFrame
documents, uuids = create_documents(df)

try:
    elastic_vector_search.add_documents(documents=documents, ids=uuids,)
    print("Documents have been indexed successfully.")
except Exception as e:
    print(f"An error occurred: {e}")


An error occurred: NotFoundError(404, 'resource_not_found_exception', 'Could not find trained model [sentence_transformers/all-MiniLM-L6-v2]')


In [43]:
from uuid import uuid4
from langchain_openai import OpenAIEmbeddings
from langchain_elasticsearch import ElasticsearchStore
from langchain_core.documents import Document
from elasticsearch import Elasticsearch



ELASTICSEARCH_URL = "http://localhost:9200"
INDEX_NAME = "langchain_index"

# Initialize OpenAI embeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Configure ElasticsearchStore
elastic_vector_search = ElasticsearchStore(
    es_url=ELASTICSEARCH_URL,
    index_name=INDEX_NAME,
    #embedding=embeddings,
    es_user="elastic",
    es_password="changeme",
    strategy=ElasticsearchStore.ApproxRetrievalStrategy(
        query_model_id="sentence-transformers__all-minilm-l6-v2"
    ),
    )



# Create Document objects
documents = []
uuids = []

for _, row in df.iterrows():
    doc_id = str(uuid4())
    documents.append(Document(
        page_content=row["review"],  # Use the "review" column for content
        metadata={
            "review_id": row["review_id"],
            "visit_id": row["visit_id"],
            "physician_name": row["physician_name"],
            "hospital_name": row["hospital_name"],
            "patient_name": row["patient_name"]
        }
    ))
    uuids.append(doc_id)

results = elastic_vector_search.add_documents(documents=documents, ids=uuids)


In [ ]:
# Create a list of documents from the DataFrame
documents, uuids = create_documents(df)

try:
    elastic_vector_search.add_documents(documents=documents, ids=uuids,)
    print("Documents have been indexed successfully.")
except Exception as e:
    print(f"An error occurred: {e}")

In [40]:
import logging

logging.basicConfig(filename='elasticsearch_errors.log', level=logging.ERROR)

try:
    results = elastic_vector_search.add_documents(documents=documents, ids=uuids)
    print("Documents have been indexed successfully.")
except Exception as e:
    logging.error(f"An error occurred: {str(e)}", exc_info=True)
    if hasattr(e, 'errors'):
        for error in e.errors:
            logging.error(error)


500 document(s) failed to index.


In [3]:
from elasticsearch import Elasticsearch

# Set up the connection to Elasticsearch
es_client = Elasticsearch(
    hosts=["http://localhost:9200"],  # Replace with your Elasticsearch host
    http_auth=("elastic", "changeme")  # Replace with your username and password if necessary
)

# Get the list of all indexes
indexes = es_client.indices.get_alias(index="*")

# Get the total number of indexes
num_indexes = len(indexes)

print(f"There are {num_indexes} indexes in Elasticsearch.")

# Optionally, you can print out the names of all indexes
for index_name in indexes:
    print(index_name)


There are 0 indexes in Elasticsearch.


/tmp/ipykernel_39216/2318990140.py:4: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es_client = Elasticsearch(


In [4]:
es_client.indices.delete(index=INDEX_NAME, ignore_unavailable=True)

ObjectApiResponse({'acknowledged': True})